In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(1, 11, (train_size, num_vars)).astype(np.float32)
p_test = np.random.uniform(1, 11, (test_size, num_vars)).astype(np.float32)
p_dev = np.random.uniform(1, 11, (val_size, num_vars)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train}, name="train")
data_test = DictDataset({"p":p_test}, name="test")
data_dev = DictDataset({"p":p_dev}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probQuadratic

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probQuadratic(x_bar, p, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probQuadratic(x_rnd, p, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p"], ["x_bar"], name="smap")

    # penalty loss for mapping
    components = [sol_map]
    loss = nm.loss.PenaltyLoss(obj_bar, constrs_bar)
    problem = nm.problem.Problem(components, loss)

    # penalty loss for rounding
    components = [sol_map, round_module]
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem_rnd = nm.problem.Problem(components, loss)

    return problem, problem_rnd

## Exact Solver

In [8]:
from problem.solver import exactQuadratic
model = exactQuadratic(n_vars=num_vars, n_integers=num_ints)

In [9]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    model.setParamValue(*p)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:24<00:00, 11.85it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    200.432000          3.835546      0.083735
std      78.224654          5.574005      0.034054
min      36.000000          0.000000      0.057559
25%     149.000000          0.000000      0.061650
50%     190.000000          0.000000      0.075394
75%     247.000000          8.126751      0.076709
max     511.000000         25.352531      0.228122


In [10]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"[-1.0, 8.0, -1.0, 8.0, 2.0]",134.0,0.000000,0.117057
1,"[-3.0, 7.0, 0.0, 8.0, 1.0]",123.0,0.000000,0.074816
2,"[-6.0, 8.0, 1.0, 10.0, 4.0]",217.0,0.000000,0.076735
3,"[-2.0, 12.0, 1.0, 9.0, 6.0]",266.0,0.000000,0.075107
4,"[-3.0, 7.0, -3.0, 8.0, -1.0]",132.0,0.000000,0.075651
...,...,...,...,...
995,"[-6.0, 11.0, -2.0, 13.0, 3.0]",339.0,17.149001,0.060144
996,"[-1.0, 11.0, 1.0, 9.0, 4.0]",220.0,0.000000,0.077659
997,"[-1.0, 11.0, 1.0, 9.0, 4.0]",220.0,5.488008,0.074160
998,"[-1.0, 11.0, 1.0, 9.0, 4.0]",220.0,9.041870,0.062052


## Heuristic

In [11]:
from heuristic import naive_round

In [12]:
# relaxed model
model_rel = model.relax()

In [13]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    model_rel.setParamValue(*p)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:02<00:00,  8.15it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  1000.00000       1000.000000   1000.000000
mean    179.96800         28.608444      0.121861
std      76.37923          7.851483      0.044695
min      26.00000         10.634741      0.058464
25%     130.00000         21.912093      0.091372
50%     170.00000         27.912093      0.121078
75%     222.00000         33.912093      0.135634
max     471.00000         53.912093      0.290110


## Learning to Round

In [14]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
_, problem = getNMProb(round_func)

In [15]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 937.0948486328125
epoch: 1  train_loss: 402.47930908203125
epoch: 2  train_loss: 381.07696533203125
epoch: 3  train_loss: 372.87158203125
epoch: 4  train_loss: 380.60809326171875
epoch: 5  train_loss: 380.5409240722656
epoch: 6  train_loss: 382.86431884765625
epoch: 7  train_loss: 377.9921875
epoch: 8  train_loss: 372.2610168457031
epoch: 9  train_loss: 370.8432922363281
epoch: 10  train_loss: 370.6485595703125
epoch: 11  train_loss: 361.46246337890625
epoch: 12  train_loss: 358.6501770019531
epoch: 13  train_loss: 364.6545104980469
epoch: 14  train_loss: 356.6761474609375
epoch: 15  train_loss: 355.5833740234375
epoch: 16  train_loss: 351.9570007324219
epoch: 17  train_loss: 357.2872009277344
epoch: 18  train_loss: 349.68865966796875
epoch: 19  train_loss: 350.5256042480469
epoch: 20  train_loss: 351.1717529296875
epoch: 21  train_loss: 346.0594482421875
epoch: 22  train_loss: 348.74774169921875
epoch: 23  train_loss: 348.5304260253906
epoch: 24  train_loss: 346.

In [16]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 234.12it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    201.908000          8.299482      0.003886
std      68.289846          2.787089      0.000768
min      47.000000          2.687708      0.002000
25%     153.000000          6.265262      0.003503
50%     194.000000          8.012799      0.003974
75%     247.000000         10.265262      0.004235
max     471.000000         18.542614      0.007208


## Learnable Threshold

In [17]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.intInd}, name="round")
_, problem = getNMProb(round_func)

In [18]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 975.09033203125
epoch: 1  train_loss: 392.9432067871094
epoch: 2  train_loss: 355.19049072265625
epoch: 3  train_loss: 352.5472106933594
epoch: 4  train_loss: 347.5971984863281
epoch: 5  train_loss: 347.3907165527344
epoch: 6  train_loss: 346.7637634277344
epoch: 7  train_loss: 340.3681945800781
epoch: 8  train_loss: 342.6441955566406
epoch: 9  train_loss: 342.302978515625
epoch: 10  train_loss: 339.9844055175781
epoch: 11  train_loss: 340.7442321777344
epoch: 12  train_loss: 339.2669677734375
epoch: 13  train_loss: 339.19537353515625
epoch: 14  train_loss: 338.64849853515625
epoch: 15  train_loss: 337.8860168457031
epoch: 16  train_loss: 338.0227355957031
epoch: 17  train_loss: 337.99822998046875
epoch: 18  train_loss: 340.48162841796875
epoch: 19  train_loss: 337.3781433105469
epoch: 20  train_loss: 337.25927734375
epoch: 21  train_loss: 335.99505615234375
epoch: 22  train_loss: 337.12945556640625
epoch: 23  train_loss: 338.3099670410156
epoch: 24  train_loss: 3

In [19]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 213.29it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    207.073000          8.717267      0.004213
std      73.458327          3.018748      0.001014
min      40.000000          2.650724      0.002000
25%     156.000000          6.302247      0.003510
50%     198.000000          8.302247      0.004001
75%     255.000000         10.542614      0.004998
max     500.000000         20.109414      0.007771


## Learning to Round with Fixed Solution Mapping

In [20]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.intInd}, name="round")
problem_rel, problem_rnd = getNMProb(round_func)

In [21]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 932.9334106445312
epoch: 1  train_loss: 356.8347473144531
epoch: 2  train_loss: 336.45758056640625
epoch: 3  train_loss: 331.31048583984375
epoch: 4  train_loss: 324.2931213378906
epoch: 5  train_loss: 318.75872802734375
epoch: 6  train_loss: 316.02508544921875
epoch: 7  train_loss: 313.67919921875
epoch: 8  train_loss: 313.3650817871094
epoch: 9  train_loss: 312.2769775390625
epoch: 10  train_loss: 312.3884582519531
epoch: 11  train_loss: 311.3890686035156
epoch: 12  train_loss: 310.96063232421875
epoch: 13  train_loss: 312.8399353027344
epoch: 14  train_loss: 309.7454528808594
epoch: 15  train_loss: 312.87237548828125
epoch: 16  train_loss: 309.1521301269531
epoch: 17  train_loss: 309.9900817871094
epoch: 18  train_loss: 310.71148681640625
epoch: 19  train_loss: 306.4747619628906
epoch: 20  train_loss: 307.1275634765625
epoch: 21  train_loss: 307.9350891113281
epoch: 22  train_loss: 306.3865661621094
epoch: 23  train_loss: 307.3204345703125
epoch: 24  train_loss

In [22]:
# freeze sol mapping
#problem_rel.freeze()
# training for rounding
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rnd, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 395.63311767578125
epoch: 1  train_loss: 395.4594421386719
epoch: 2  train_loss: 395.5500793457031
epoch: 3  train_loss: 394.0802307128906
epoch: 4  train_loss: 395.0129699707031
epoch: 5  train_loss: 396.0818176269531
epoch: 6  train_loss: 395.1732482910156
epoch: 7  train_loss: 395.8547668457031
epoch: 8  train_loss: 395.42840576171875
epoch: 9  train_loss: 395.6715087890625
epoch: 10  train_loss: 395.98187255859375
epoch: 11  train_loss: 395.4700927734375
epoch: 12  train_loss: 395.11761474609375
epoch: 13  train_loss: 397.1585388183594
epoch: 14  train_loss: 396.5810852050781
epoch: 15  train_loss: 394.6093444824219
epoch: 16  train_loss: 394.5400695800781
epoch: 17  train_loss: 394.8847961425781
epoch: 18  train_loss: 395.0920104980469
epoch: 19  train_loss: 396.547119140625
epoch: 20  train_loss: 395.2701110839844
epoch: 21  train_loss: 395.3181457519531
epoch: 22  train_loss: 396.2032165527344
epoch: 23  train_loss: 397.4877014160156
epoch: 24  train_loss: 

In [23]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_rnd(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 229.03it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    189.095000          9.041808      0.003920
std      72.065911          2.930551      0.000862
min      27.000000          2.735447      0.001999
25%     136.000000          6.887273      0.003317
50%     183.500000          8.585647      0.003999
75%     234.000000         10.841309      0.004416
max     443.000000         18.579599      0.008525


### Learning to Feasibility Pump

In [24]:
from model.layer import netFC
# define neural architecture for the solution mapping
sol_func = nm.modules.blocks.MLP(insize=num_vars, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU,
                                 hsizes=[80]*4)
# define neural architecture for rounding
rnd_layer = netFC(input_dim=num_vars*2, hidden_dims=[80]*4, output_dim=num_vars)

In [25]:
# function to get nm optimization model
from problem.neural import probQuadratic
def getProb(x, p):
    return probQuadratic(x, p, num_vars=num_vars, alpha=100)

In [26]:
from model import feasibilityPumpModel
# feasibility pump model
problem_rel, problem_fp = feasibilityPumpModel(["p"], getProb, sol_func, rnd_layer, int_ind=model.intInd, num_iters=3)

In [27]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 1055.1697998046875
epoch: 1  train_loss: 375.1524963378906
epoch: 2  train_loss: 328.26104736328125
epoch: 3  train_loss: 321.15948486328125
epoch: 4  train_loss: 317.2909240722656
epoch: 5  train_loss: 315.58453369140625
epoch: 6  train_loss: 314.23565673828125
epoch: 7  train_loss: 312.3757019042969
epoch: 8  train_loss: 311.7950744628906
epoch: 9  train_loss: 311.66888427734375
epoch: 10  train_loss: 310.39910888671875
epoch: 11  train_loss: 310.2560729980469
epoch: 12  train_loss: 310.1974182128906
epoch: 13  train_loss: 309.4531555175781
epoch: 14  train_loss: 309.32568359375
epoch: 15  train_loss: 308.2574462890625
epoch: 16  train_loss: 309.0048522949219
epoch: 17  train_loss: 308.4734802246094
epoch: 18  train_loss: 308.40618896484375
epoch: 19  train_loss: 307.931640625
epoch: 20  train_loss: 306.3957214355469
epoch: 21  train_loss: 305.8752136230469
epoch: 22  train_loss: 305.4753723144531
epoch: 23  train_loss: 306.5816955566406
epoch: 24  train_loss: 3

In [28]:
# freeze sol mapping
#problem_rel.freeze()
# training for feasibility pump
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_fp, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 2151.41552734375
epoch: 1  train_loss: 2151.20361328125
epoch: 2  train_loss: 2150.4990234375
epoch: 3  train_loss: 2155.776611328125
epoch: 4  train_loss: 2151.611328125
epoch: 5  train_loss: 2152.010009765625
epoch: 6  train_loss: 2148.83935546875
epoch: 7  train_loss: 2147.4033203125
epoch: 8  train_loss: 2152.97021484375
epoch: 9  train_loss: 2155.150146484375
epoch: 10  train_loss: 2145.474853515625
epoch: 11  train_loss: 2153.859619140625
epoch: 12  train_loss: 2154.91357421875
epoch: 13  train_loss: 2149.227783203125
epoch: 14  train_loss: 2153.484130859375
epoch: 15  train_loss: 2148.620849609375
epoch: 16  train_loss: 2149.673095703125
epoch: 17  train_loss: 2153.195068359375
epoch: 18  train_loss: 2150.64501953125
epoch: 19  train_loss: 2147.54931640625
epoch: 20  train_loss: 2146.578857421875
epoch: 21  train_loss: 2148.906005859375
epoch: 22  train_loss: 2148.907470703125
epoch: 23  train_loss: 2152.666015625
epoch: 24  train_loss: 2149.102294921875
ep

In [29]:
sols, objvals, conviols, elapseds = [], [], [], []
for p in tqdm(p_test):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 218.08it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    207.073000          8.717267      0.004109
std      73.458327          3.018748      0.000835
min      40.000000          2.650724      0.002000
25%     156.000000          6.302247      0.003512
50%     198.000000          8.302247      0.004004
75%     255.000000         10.542614      0.004564
max     500.000000         20.109414      0.007529
